In [1]:
import pandas as pd 
data=pd.read_csv(r'C:\Users\Varun Kurup\Documents\rnnshortdataset.csv')

In [2]:
import re, nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

def normalizer(data):
    only_letters = re.sub("[^a-zA-Z]", " ",data) 
    tokens = nltk.word_tokenize(only_letters)[2:]
    lower_case = [l.lower() for l in tokens]
    filtered_result = list(filter(lambda l: l not in stop_words, lower_case))
    lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
    return lemmas

In [3]:
import nltk
nltk.download('wordnet')

normalizer("Here is the email")

[nltk_data] Downloading package wordnet to C:\Users\Varun
[nltk_data]     Kurup\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['email']

In [4]:
pd.set_option('display.max_colwidth', -1) # Setting this so we can see the full content of cells
data['normalized_data'] = data.iloc[:,0].apply(normalizer)
data[['Label','normalized_data']].head()

,Label,normalized_data
0,2,[confirmed]
1,0,[confirmed]
2,0,[rejected]
3,2,[accepted]
4,2,[]


In [5]:
from nltk import ngrams
def ngrams(input_list):
    #onegrams = input_list
    bigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:]))]
    trigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:], input_list[2:]))]
    return bigrams+trigrams
data['grams'] = data.normalized_data.apply(ngrams)
data[['grams']].head()

,grams
0,[]
1,[]
2,[]
3,[]
4,[]


In [6]:
import collections
def count_words(input):
    cnt = collections.Counter()
    for row in input:
        for word in row:
            cnt[word] += 1
    return cnt

In [7]:
import numpy as np
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(ngram_range=(1,2))

In [8]:
vectorized_data = count_vectorizer.fit_transform(data.iloc[:,0])
indexed_data = hstack((np.array(range(0,vectorized_data.shape[0]))[:,None], vectorized_data))

In [9]:
from sklearn.model_selection import train_test_split
data_train, data_test, targets_train, targets_test = train_test_split(indexed_data, data.Label, test_size=0.3, random_state=0)
data_train_index = data_train[:,0]
data_train = data_train[:,1:]
data_test_index = data_test[:,0]
data_test = data_test[:,1:]

In [10]:
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
clf_output = clf.fit(data_train, targets_train)

In [11]:
clf.score(data_test, targets_test)

0.7

In [98]:
print(targets_test)

84     2
10     0
75     2
2      0
24     0
100    0
107    2
7      2
16     2
86     2
68     2
22     0
45     2
60     0
76     2
52     2
13     2
73     0
85     0
54     0
103    1
8      0
Name: Label, dtype: int64


In [12]:
from sklearn.metrics import classification_report
preds=clf.predict(data_test)
print(classification_report(targets_test,preds))

              precision    recall  f1-score   support

           0       0.80      0.36      0.50        11
           1       0.50      0.67      0.57         3
           2       0.71      0.94      0.81        16

   micro avg       0.70      0.70      0.70        30
   macro avg       0.67      0.66      0.63        30
weighted avg       0.72      0.70      0.67        30

